# Thanks Again Sander!

# Table of Contents
 <p><div class="lev1"><a href="#Plot-functions"><span class="toc-item-num">1 - </span>Plot functions</a></div><div class="lev2"><a href="#General-plot-options"><span class="toc-item-num">1.1 - </span>General plot options</a></div><div class="lev2"><a href="#Scatter-plots"><span class="toc-item-num">1.2 - </span>Scatter plots</a></div><div class="lev2"><a href="#Histograms"><span class="toc-item-num">1.3 - </span>Histograms</a></div><div class="lev3"><a href="#Using-a-more-powerful-hist-package"><span class="toc-item-num">1.3.1 - </span>Using a more powerful hist package</a></div><div class="lev2"><a href="#For-plotting-PMT-arrays"><span class="toc-item-num">1.4 - </span>For plotting PMT arrays</a></div><div class="lev2"><a href="#Fit-functions"><span class="toc-item-num">1.5 - </span>Fit functions</a></div><div class="lev3"><a href="#How-to-change-font-in-plots"><span class="toc-item-num">1.5.1 - </span>How to change font in plots</a></div>

# Plot functions

[Here we put some plot functions that we use more often]

## General plot options

Some plot functions that are used by other functions

In [ ]:
# Saves plots to notebookname/YYMMDD/...
plot_folder = os.path.join(notebook_name, strftime("%Y%m%d"))
plot_folder_web = os.path.join( plot_folder, "web")

In [ ]:
# changes the general size of the plots
size_multiplier = 1

# function makes folder, pdf, png and web png (small)
def save_plot(title):
    if not os.path.exists(plot_folder):
        os.makedirs(plot_folder)
    if not os.path.exists(plot_folder_web):
        os.makedirs(plot_folder_web)
    if save_pdf_plots:
        plt.savefig(os.path.join(plot_folder, title) + '.pdf', format='pdf')
    if save_plots:
        plt.savefig(os.path.join(plot_folder, title) + '.png', format='png', dpi=100)
        plt.savefig(os.path.join(plot_folder_web, title) + '.png', format='png', dpi=50)

# function to set the axis
def set_axis(xaxis,yaxis,title):
    plt.xlabel(xaxis, fontsize = 20)
    plt.ylabel(yaxis, fontsize = 20)
    plt.title(title, fontsize = 24, y =1.03)
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.tick_params(axis='both', which='minor', labelsize=20)

## Scatter plots

In [ ]:
# actual scatter plotting function that uses set_axis and save_plot
def plot_scatter(x,y,xtitle,ytitle,title,xlim=[None,None],ylim=[None,None], colorb = False, 
                 colorblabel=None, discrete = False, vmax = None, logscale=False, grid = True, *args, **kwargs):
    if discrete:
        plt.scatter(x,y,vmax = vmax,cmap = plt.get_cmap('jet', vmax), *args, **kwargs)
    else:
        plt.scatter(x,y,vmax = vmax, *args, **kwargs)
    plt.xlim(xlim)
    plt.ylim(ylim)
    set_axis(xtitle,ytitle,title)
    if colorb:
        if discrete == True:
            tick_bounds = np.linspace(0,vmax,vmax+1)
            tickpos = np.linspace(tick_bounds[0] + .5, tick_bounds[-1] - .5, len(tick_bounds)-1)
            cax = plt.colorbar(label=colorblabel, ticks=tickpos)
            cax.set_ticklabels(tick_bounds)
        else:
            plt.colorbar(label=colorblabel)
    if logscale:
        plt.xscale('log')
        plt.yscale('log')
    save_plot(title)
    if grid:
        plt.grid()
    plt.show()
 


## Histograms

In [1]:
# function for getting bin centers for histograms
def bin_centers(bin_edges):
    return 0.5*(bin_edges[1:] + bin_edges[:-1])

# function for 2D histogram
def twod_stat_plot(x, y, z, statistic='mean', bins=10, range=None, vrange=None, cblabel=None):
    if vrange is None:
        vrange = (None, None)
    result, xbinedges, ybinedges, binnumbers = binned_statistic_2d(
        x, y, z,
        bins=bins,
        statistic=statistic,
        range=range
    )
    xx, yy = np.meshgrid(xbinedges, ybinedges)    

    Zm = np.ma.masked_where(np.isnan(result),result)
    plt.figure(figsize=(size_multiplier*12, size_multiplier*10))
    plt.pcolormesh(xx, yy, Zm.T, vmin=vrange[0], vmax=vrange[1])
    plt.colorbar(label=cblabel)
    
# Function to make the colorbar in discrete steps
def discrete_matshow(data):
    cmap = plt.get_cmap('RdBu', np.max(data)-np.min(data)+1)
    mat = plt.matshow(data,cmap=cmap)
    ticks = np.arange(np.min(data),np.max(data)+1)
    tickpos = np.linspace(ticks[0] + .5, ticks[-1] - .5, len(ticks));
    cax = plt.colorbar(mat, ticks=tickpos)
    cax.set_ticklabels(ticks)

### Using a more powerful hist package 

This uses Jelle's multihist package

In [ ]:
from multihist import *

def plot_2dhist(x,y,xtitle,ytitle,title,rangexy = [[None,None],[None,None]], colorb = False, 
                 colorblabel=None, bins =[100,100],vmin=None, vmax = None, *args, **kwargs):
    m2 = Histdd(bins=bins, range=rangexy,*args, **kwargs)
    m2.add(x,y)
    m2.plot(cblabel = colorblabel, vmin=vmin, vmax=vmax)
    set_axis(xtitle,ytitle,title)
    save_plot(title)
    plt.show()

In [ ]:
### code for three plots at the same time:
### builds one 2dhist with two 1d histograms on the axis

# overwrite fielddata in the analysis notebook

# fielddata = {
#     'defaultname':                       {'range':(0,999),       'descr':"default units"}, 
#     }

def getfieldtitle(key):
    for fieldname,fd in fielddata.items():
        if not 'descr' in fd: continue
        if key==fieldname: return fd['descr']
    return key

def getsanerange(data, key, m = 5.):
    stuff = data[key]
    (mi,ma) = (None,None)
    #Are there any predefined minima and maxima?
    for fn,_ in fielddata.items():
        if key==fn:
        #if re.match('(.*)%s'%fn,key):   #use settings for blah if key is LhsOM_JdGYGFNblah
                if 'range' in fielddata[fn]:
                    (mi,ma) = fielddata[fn]['range']
                if mi=='All': mi = np.min(stuff)
                if ma=='All': ma = np.max(stuff)
    #Define minima and maxima to exclude outliers
    median = np.median(stuff)
    mediandev = np.median(np.abs(stuff - median))
    if mi==None: mi = max(np.min(stuff),median-m*mediandev)
    if ma==None: ma = min(np.max(stuff),median+m*mediandev)
    return (mi,ma)

# def make_densityhistogram(data, xkey, ykey, title="Very nice plot!", bins=100, logscale=False): 
def make_densityhistogram(x, y, xkey, ykey, title="Very nice plot!", bins=100, logscale=False): 
    data = {xkey : x, ykey : y}
    print("\t" + title) 
    #Stolen and modified from http://www.astrobetter.com/visualization-fun-with-python-2d-histogram-with-1d-histograms-on-axes/ 
    xlims  = getsanerange(data, xkey) 
    ylims  = getsanerange(data, ykey) 
    x = data[xkey] 
    y = data[ykey] 
    xlabel = getfieldtitle(xkey) 
    ylabel = getfieldtitle(ykey) 
     
    # Define the locations for the axes             #??? 
    left, width = 0.12, 0.55 
    bottom, height = 0.12, 0.55 
    bottom_h = left_h = left+width+0.02 
      
    # Set up the geometry of the three plots 
    rect_temperature = [left, bottom, width, height] # dimensions of temp plot 
    rect_histx = [left, bottom_h, width, 0.25] # dimensions of x-histogram 
    rect_histy = [left_h, bottom, 0.25, height] # dimensions of y-histogram 
      
    # Set up the size of the figure 
    plt.close() 
    fig = plt.figure(1, figsize=(9.5,9)) 
      
    # Make the three plots 
    axTemperature = plt.axes(rect_temperature) # temperature plot 
    axHistx = plt.axes(rect_histx) # x histogram 
    axHisty = plt.axes(rect_histy) # y histogram 
      
    # Remove the inner axes numbers of the histograms 
    from matplotlib.ticker import NullFormatter 
    nullfmt = NullFormatter() 
    axHistx.xaxis.set_major_formatter(nullfmt) 
    axHisty.yaxis.set_major_formatter(nullfmt) 
      
    # Find the min/max of the data 
    xmin = min(xlims) 
    xmax = max(xlims) 
    ymin = min(ylims) 
    ymax = max(ylims) 
      
    # Make the 'main' temperature plot 
    # Define the number of bins 
    nxbins = bins 
    nybins = bins 
    nbins = bins 
      
    xbins = np.linspace(start = xmin, stop = xmax, num = nxbins) 
    ybins = np.linspace(start = ymin, stop = ymax, num = nybins) 
    xcenter = (xbins[0:-1]+xbins[1:])/2.0 
    ycenter = (ybins[0:-1]+ybins[1:])/2.0 
    aspectratio = 1.0*(xmax - xmin)/(1.0*ymax - ymin) 
    #aspectratio = 1.0*(xmax - 0)/(1.0*ymax - 0)    Weird stuff with one large range, one negative range value 
 
    H, xedges,yedges = np.histogram2d(y,x,bins=(ybins,xbins)) 
    X = xcenter 
    Y = ycenter 
    Z = H 
      
    # Plot the temperature data 
    #coudl set cmap=plt.cm.hot... doesn't make stuff clearer though 
    cax = (axTemperature.imshow(H, extent=[xmin,xmax,ymin,ymax], 
           interpolation='nearest', origin='lower',aspect=aspectratio)) 
      
    #Plot the axes labels 
    axTemperature.set_xlabel(xlabel) 
    axTemperature.set_ylabel(ylabel) 
      
    #Set up the plot limits 
    axTemperature.set_xlim(xlims) 
    axTemperature.set_ylim(ylims) 
      
    #Set up the histogram bins 
    xbins = np.arange(xmin, xmax, (xmax-xmin)/nbins) 
    ybins = np.arange(ymin, ymax, (ymax-ymin)/nbins) 
      
    #Plot the histograms 
    axHistx.hist(x, bins=xbins, color='blue', log=logscale) 
    axHisty.hist(y, bins=ybins, orientation='horizontal', color='red', log=logscale) 
      
    #Set up the histogram limits 
    axHistx.set_xlim( min(xlims), max(xlims) ) 
    axHisty.set_ylim( min(ylims), max(ylims) ) 
     
    #Make the tickmarks pretty 
    ticklabels = axHistx.get_yticklabels() 
    for label in ticklabels: 
        label.set_fontsize(6) 
        #label.set_family('serif') 
  
    #Make the tickmarks pretty 
    ticklabels = axHisty.get_xticklabels() 
    for label in ticklabels: 
        label.set_fontsize(6) 
        #label.set_family('serif') 
     
    #Done! 
    plt.suptitle(title) 
    save_plot(title)

## For plotting PMT arrays

In [1]:
## For PMT arrays
def plot_pmts(pmt_range,  s=20, colorbar =True, **kwargs):
    plt.scatter([sim.config['pmt_locations'][i]['x'] for i in pmt_range], 
                [sim.config['pmt_locations'][i]['y'] for i in pmt_range],
                 s=s, edgecolors='white', **kwargs)
    if colorbar:
        plt.colorbar(label = 'Light (phd)')
    
def plot_top_array_dead():
    plot_pmts([ch for ch in sim.config['channels_top'] if ch in dead_pmts and ch != 0],
              s=40, c='red', marker = 's', colorbar =False)
    
def plot_bottom_array_dead():
    plot_pmts([ch for ch in sim.config['channels_bottom'] if ch in dead_pmts and ch != 0],
              s=40, c='red', marker = 's', colorbar =False)

def plot_top_saturated(saturated_channel_numbers):
    plot_pmts([ch for ch in saturated_channel_numbers if ch in sim.config['channels_top']],
               s = 20, c = 'black', marker = 'D', colorbar =False)
    
def plot_bottom_saturated(saturated_channel_numbers):
    plot_pmts([ch for ch in saturated_channel_numbers if ch in sim.config['channels_bottom']],
               s = 20, c = 'black', marker = 'D', colorbar =False)

def plot_top_array( **kwargs):
    plot_pmts([ch for ch in sim.config['channels_top'] if ch not in dead_pmts],
              s=20, c='b' ,marker = 'o', **kwargs)
    
def plot_bottom_array( **kwargs):
    plot_pmts([ch for ch in sim.config['channels_bottom'] if ch not in dead_pmts],
              s=20, c='b' ,marker = 'o', **kwargs)

def plot_detector_radius():
    plt.xlim((-1.1*detector_radius, 1.1*detector_radius))
    plt.ylim((-1.1*detector_radius, 1.1*detector_radius))

    plt.gca().add_artist(plt.Circle((0,0), 
                                    detector_radius, 
                                    edgecolor='black', 
                                    fill=None))
    plt.gca().add_artist(plt.Circle((0,0), 
                                    fiducial_volume_radius, 
                                    edgecolor='0.7',
                                    fill=None))

## Fit functions

In [ ]:
def gaussian(x,x_0,sigma,a):
    return a*np.exp(-(x-x_0)**2/ (2*((sigma)**2)))

def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    Stolen from http://stackoverflow.com/questions/2413522
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return average, np.sqrt(variance)


def fit_gaussian_hist(hist, sigma_percent):
    # Use indexing rather than unpacking to support both return value from plt.hist an np.histogram
    histogram = hist[0]
    bin_edges = hist[1]
    bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])
    mean_g, sigma_g = weighted_avg_and_std(bin_centers, weights=histogram)
    a_g = histogram.sum() / np.sqrt(2*np.pi*sigma_g**2)
    popt,pcov = curve_fit(f=gaussian,
                          xdata=bin_centers,
                          ydata=histogram,
                          p0=[mean_g,sigma_g,a_g,],
                          sigma=np.clip(np.sqrt(histogram), 1, float('inf')))
    plt.plot(bin_edges, gaussian(bin_edges, *popt), 'r', label = 'Gaussian fit')
    return popt,pcov

## Drawing a box in a plot

In [ ]:
def draw_box(x, y, **kwargs):
    """Draw rectangle, given x-y boundary tuples"""
    # Arcane syntax of the week: matplotlib's Rectangle...
    plt.gca().add_patch(matplotlib.patches.Rectangle(
        (x[0], y[0]), x[1] - x[0], y[1] - y[0], facecolor='none', **kwargs))

### How to change font in plots

Just some instructions on how to change the font of the text in your plots

In [ ]:
# 1. Install arial on your system
# sudo apt-get install fonts-liberation
# sudo apt-get install msttcorefonts
# sudo fc-cache -f -v

# 2. delete your matplotlib cash in ~/.matplotlib or ~/.cache/matplotlib

# 3. In your code import matplotlib and set font to Arial
# (import matplotlib.pyplot as plt)

# plt.rcParams.update({'font.sans-serif':'Arial'})

# To check:
# fc-list | grep -i "Arial"

# Cheers, you now have another font for publishing in papers